In [1]:
import os
from datetime import datetime
from pathlib import Path

from core.runner import Runner
from core.environment import SingleVertexSpinFoam, StarModelSpinFoam


def train_gfn(
    spin_j,
    sf_model,
    main_layer_hidden_nodes,
    branch1_hidden_nodes,
    branch2_hidden_nodes,
    activation_function,
    exploration_rate,
    training_fraction_from_back_traj,
    learning_rate,
    batch_size,
    n_iterations,
    evaluation_batch_size,
    generate_samples_every_m_training_samples,
):
    if sf_model == "single_vertex_model":
        spinfoam_model = SingleVertexSpinFoam(spin_j=spin_j)
    elif sf_model == "star_model":
        spinfoam_model = StarModelSpinFoam(spin_j=spin_j)
    else:
        raise ValueError(
            "Spinfoam model not yet implemented. " "Custom Spinfoam class can be made."
        )

    # training_run_datetime = datetime.now().strftime("%B %d, %Y at %H:%M:%S")

    base_data_folder = Path(
        f"./data/",
        f"GFlowNet/{sf_model}/j_{spin_j}/",
        f"main_layer_hid_nodes_{main_layer_hidden_nodes}/",
        f"exploration_rate_{exploration_rate}/learning_rate_{learning_rate}/activation_function_{activation_function}",
        f"n_iterations_{n_iterations}/evaluation_batch_size_{evaluation_batch_size}/batch_size_{batch_size}"
    )

    runner = Runner(
        spinfoam_model=spinfoam_model,
        main_layer_hidden_nodes=main_layer_hidden_nodes,
        branch1_hidden_nodes=branch1_hidden_nodes,
        branch2_hidden_nodes=branch2_hidden_nodes,
        activation=activation_function,
        exploration_rate=exploration_rate,
        training_fraction_from_back_traj=training_fraction_from_back_traj,
        learning_rate=learning_rate,
    )

    ave_losses = runner.train_agent(
        batch_size,
        n_iterations,
        evaluation_batch_size,
        generate_samples_every_m_training_samples,
        base_data_folder,
    )

    return ave_losses

2023-05-12 14:42:14.746431: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-12 14:42:15.191526: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-12 14:42:16.185357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/frisus/Scrivania/sl2cfoam-next-dev/lib:/home/frisus/anaconda3/envs/tf/lib/
2023-05-12 14:42:16.185434: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvi

## Vertex Amplitude

In [8]:
spin_j = 2.0
sf_model = "single_vertex_model"
main_layer_hidden_nodes = (30, 20)
branch1_hidden_nodes = ()
branch2_hidden_nodes = ()
activation = "swish"
exploration_rate = 0.75
training_fraction_from_back_traj = 0.0
learning_rate = 0.001
batch_size = int(1e3)
n_iterations = int(1e4)
evaluation_batch_size = int(1e5)
generate_samples_every_m_training_samples = int(1e6)

In [9]:
ave_losses = train_gfn(
    spin_j,
    sf_model,
    main_layer_hidden_nodes,
    branch1_hidden_nodes,
    branch2_hidden_nodes,
    activation,
    exploration_rate,
    training_fraction_from_back_traj,
    learning_rate,
    batch_size,
    n_iterations,
    evaluation_batch_size,
    generate_samples_every_m_training_samples,
)

Nth iteration: 1000 Trained on K samples: 1000000 Average Loss: 1.8019597771625295
Nth iteration: 2000 Trained on K samples: 2000000 Average Loss: 1.089730833543276
Nth iteration: 3000 Trained on K samples: 3000000 Average Loss: 0.70374261392878545
Nth iteration: 4000 Trained on K samples: 4000000 Average Loss: 0.52576329902285013
Nth iteration: 5000 Trained on K samples: 5000000 Average Loss: 0.59975175351802756
Nth iteration: 6000 Trained on K samples: 6000000 Average Loss: 0.47773070721714606
Nth iteration: 7000 Trained on K samples: 7000000 Average Loss: 0.41230756319826706
Nth iteration: 8000 Trained on K samples: 8000000 Average Loss: 0.353062084619325
Nth iteration: 9000 Trained on K samples: 9000000 Average Loss: 0.32625513370677822
Nth iteration: 10000 Trained on K samples: 10000000 Average Loss: 0.40590473313555725
INFO:tensorflow:Assets written to: data/GFlowNet/single_vertex_model/j_2.0/main_layer_hid_nodes_(30, 20)/exploration_rate_0.75/learning_rate_0.001/activation_funct